In [7]:
import numpy as np
import tifffile as tiff
from PIL import Image
from ultralytics import YOLO
from scipy.spatial import Voronoi
import matplotlib.pyplot as plt
from matplotlib.path import Path

def load_and_tile_image(image_path, tile_size):
    image = tiff.imread(image_path)
    tiles = []
    for i in range(0, image.shape[0], tile_size[1]):
        for j in range(0, image.shape[1], tile_size[0]):
            tile = image[i:i + tile_size[1], j:j + tile_size[0]]
            if tile.shape[0] != tile_size[1] or tile.shape[1] != tile_size[0]:
                # Pad the tile if it does not meet the expected dimensions
                tile = np.pad(tile, [(0, max(0, tile_size[1] - tile.shape[0])), 
                                     (0, max(0, tile_size[0] - tile.shape[1]))], 
                              mode='constant', constant_values=0)
            tiles.append(tile)
    return tiles

def predict_cells(tile, model):
    if tile.size == 0:
        return []
    # Convert tile to RGB and add batch dimension for YOLO
    tile_image = Image.fromarray(tile).convert('RGB')
    tile_image = np.array(tile_image)[np.newaxis, ...]  # Add batch dimension
    results = model(tile_image)
    cell_centers = [( (box[0] + box[2]) / 2, (box[1] + box[3]) / 2 ) for *box, conf, cls in results.pred[0]]
    return cell_centers

def process_tile(tile, model):
    cell_centers = predict_cells(tile, model)
    if not cell_centers:
        return np.zeros(tile.shape[:2], dtype=int)
    vor = Voronoi(cell_centers)
    tile_array = np.zeros(tile.shape[:2], dtype=int)
    for region_idx, region in enumerate(vor.regions):
        if not -1 in region:
            polygon = [vor.vertices[i] for i in region if i >= 0 and i < len(vor.vertices)]
            path = Path(polygon)
            x, y = np.meshgrid(np.arange(tile_array.shape[1]), np.arange(tile_array.shape[0]))
            points = np.vstack((x.flatten(), y.flatten())).T
            grid = path.contains_points(points)
            grid = grid.reshape(tile_array.shape)
            tile_array[grid] = region_idx + 1
    return tile_array

# Load and initialize the model
model_path = r"S:\Phys\FIV925 XSection\Datasets\Brain\01c\YO 432 0515 Snow\map75=0641477  pt65 idx=2 ep=4 btch=16 rnd=7684896\weights\best.pt"
model = YOLO(model_path)

# Load and tile the image
image_path = r"S:\Phys\FIV925 XSection\Datasets\Human Lung\Infer\DAPI\DAPI_Juanru_Lung_DAPI.tif"
tiles = load_and_tile_image(image_path, tile_size=(256, 256))

# Process each tile
processed_tiles = [process_tile(tile, model) for tile in tiles]

# Optional: Visualize or reassemble tiles
for idx, tile in enumerate(processed_tiles):
    plt.figure(figsize=(5, 5))
    plt.title(f'Tile {idx+1}')
    plt.imshow(tile, cmap='jet')
    plt.colorbar()
    plt.show()




error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3789: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'
